In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('drive/My Drive/Teaching/LT_2021_2022/ST456/Week04')

from BS_Fun import *
from tf_Fun3 import *

initialisation_tf()

### Activities

* Use `Model1_make` to declare the architecture. 
* Use different gradient methods to run it: `Adagrad`, `RMSprop`, and `Adam`. 

In [ ]:
def generate_sample_no_noise(S=10,K=np.arange(5,21,1),
                             T=1,r=0,true_sigma=np.arange(0.1,5,0.01)):
    '''
    We receive data over different calls, and would like to recover sigma
    Assume that we have no noise in the data
    '''
    trial = BS_Options_Pricing_sigma(S,K,T,r,true_sigma)
    trial.make_calls()
    data = df(trial.calls).transpose()
    data.reset_index(inplace=True)
    data.columns=['sig']+K.tolist()
    return data

data = generate_sample_no_noise()

y = data.pop('sig')
X = data

feature_normaliser = layers.Normalization(input_shape=[16], axis=1,name='normaliser')
feature_normaliser.adapt(X)

In [ ]:
m1 = Model1_make(feature_normaliser)
h1, m1 = Model_fit(m1,X,y,epochs=200)

m2 = Model1_make(feature_normaliser)
m2.compile(
    optimizer=tf.optimizers.Adagrad(learning_rate=0.1, initial_accumulator_value=0.1),
    loss='mean_squared_error'
)
h2,m2 = Model_fit(m2,X,y,epochs=200)

#### Students' exercise: 

Complete the remainings by filling the blanks: do not worry about validation data for now.

In [ ]:
m3 = Model1_make(feature_normaliser)
m3.compile(...)
h3, m3 = Model_fit(m3,X,y,epochs=200)

m4 = Model1_make(feature_normaliser)
m4.compile(...)
h4, m4 = ...

#### Students' exercise: 

Observe the loss trajectory and predictions.

In [ ]:
# This is a similar exercise to the Page 27 of the Lecture Slides

def merge_histories(h1,h2,h3,h4):
    D = df({'SGD':h1.history['loss']})
    D = D.join(df({'Adagrad':h2.history['loss']}))
    D = D.join(df({'RMSprop':h3.history['loss']}))
    D = D.join(df({'Adam':h4.history['loss']}))
    return D

D_m = merge_histories(h1,h2,h3,h4)
D_m.plot(ylim=(0,10))
D_m.plot(ylim=(0,2),grid=True)

In [ ]:
MP = MakePrediction([m1,m2,m3,m4],X,y)

MP.run_basic_plot(names=['SGD','Adagrad','RMSprop','Adam'],
                 color=['b']+plt.rcParams['axes.prop_cycle'].by_key()['color'][:4])

#### Students' exercise: 

Observe the weights within each layer

In [ ]:
m1 = Model1_make(feature_normaliser,learning_rate=0.1,name='SGD')

m2 = Model1_make(feature_normaliser,name='Adagrad')
m2.compile(
    optimizer=tf.optimizers.Adagrad(learning_rate=0.1, initial_accumulator_value=0.1),
    loss='mean_squared_error'
)

m3 = Model1_make(feature_normaliser,name='RMSprop')
m3.compile(
    optimizer=tf.optimizers.RMSprop(learning_rate=0.1, rho=0.9),
    loss='mean_squared_error'
)

m4 = Model1_make(feature_normaliser,name='Adam')
m4.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1, beta_1=0.9, beta_2=0.999),
    loss='mean_squared_error'
)

# helper class (from tf_Fun3) for training and collecting results
ITAC = ImplementTrainAndCollect(X,y,[m1,m2,m3,m4])
ITAC.implement_all()

In [ ]:
ITAC.plot_one(3)
plt.legend(loc='center', bbox_to_anchor=(1.1, 0.5))
ITAC.plot_one(4)
plt.legend(loc='center', bbox_to_anchor=(1.1, 0.5))
ITAC.plot_one(5)
plt.legend(loc='center', bbox_to_anchor=(1.1, 0.5))